In [1]:
import numpy as np
import ase.io
import tqdm
from pytorch_prototype.nice_blocks import *
from pytorch_prototype.utilities import *
from matplotlib import pyplot as plt
from torch import nn
from nice.rascal_coefficients import process_structures

In /home/pozdn/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/pozdn/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/pozdn/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In /home/pozdn/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/pozdn/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylel

In [2]:
LAMBDA_MAX = 4
clebsch = ClebschGordan(LAMBDA_MAX)
HYPERS = {
    'interaction_cutoff': 6.3,
    'max_radial': 4,
    'max_angular': LAMBDA_MAX,
    'gaussian_sigma_type': 'Constant',
    'gaussian_sigma_constant': 0.05,
    'cutoff_smooth_width': 0.3,
    'radial_basis': 'GTO'
}
MAGIC_NUMBER = 10000
BATCH_SIZE = 500
structures = process_structures(ase.io.read('methane.extxyz', index='0:10'))
all_species = get_all_species(structures)

In [3]:
def split(coefficients):
    lambd_max = max(coefficients.keys())
    even, odd = {}, {}
    for lambd in range(lambd_max + 1):
        if lambd % 2 == 0:
            even[str(lambd)] = coefficients[lambd]
        else:
            odd[str(lambd)] = coefficients[lambd]
    return even, odd

coefficients_all = get_coefs(structures, HYPERS, all_species)
for key in coefficients_all.keys():
    coefficients_all[key] *= MAGIC_NUMBER
central = get_central_species(structures)
mask = central == 6

for key in coefficients_all.keys():
    coefficients_all[key] = coefficients_all[key][mask]
 
even, odd = split(coefficients_all)

In [4]:
print("even: ")
for key in even.keys():
    print(key, even[key].shape)
print("odd: ")
for key in odd.keys():
    print(key, odd[key].shape)

even: 
0 torch.Size([10, 8, 1])
2 torch.Size([10, 8, 5])
4 torch.Size([10, 8, 9])
odd: 
1 torch.Size([10, 8, 3])
3 torch.Size([10, 8, 7])


In [5]:
block = ClebschCombining(clebsch.precomputed_, LAMBDA_MAX)
result = block.forward(even, even)
for key in result.keys():
    print(key, result[key].shape)

0 torch.Size([10, 192, 1])
1 torch.Size([10, 128, 3])
2 torch.Size([10, 384, 5])
3 torch.Size([10, 256, 7])
4 torch.Size([10, 384, 9])


In [6]:
task = {}
some = [np.array([0, 2]), np.array([0, 2])]
for l1 in range(LAMBDA_MAX + 1):
    for l2 in range(LAMBDA_MAX + 1):
        for lambd in range(abs(int(l1) - int(l2)), min(int(l1) + int(l2), LAMBDA_MAX) + 1):
            key = f'{l1}_{l2}_{lambd}'
            #print(key)
            task[key] = some

In [7]:
block = ClebschCombining(clebsch.precomputed_, LAMBDA_MAX, task = task)
result = block.forward(even, even)
for key in result.keys():
    print(key, result[key].shape)

0 torch.Size([10, 6, 1])
1 torch.Size([10, 4, 3])
2 torch.Size([10, 12, 5])
3 torch.Size([10, 8, 7])
4 torch.Size([10, 12, 9])


In [8]:
class Experiment(nn.Module):
    def __init__(self):
        super(Experiment, self).__init__()
        self.register_buffer('asdf', torch.FloatTensor([1, 2, 3]))
        self.register_buffer('q', torch.IntTensor([1, 2, 3]))
    def forward(self):
        print(self.asdf)
        print(self.q)

In [9]:
block = Experiment()
block()

tensor([1., 2., 3.])
tensor([1, 2, 3], dtype=torch.int32)
